In [42]:
import sys
sys.path.append("./../..")

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import torch
import torch.nn as nn
import numpy as np

#
from effcn.models_mnist import CapsNet
from misc.utils import count_parameters

### receptive firld

In [45]:
def rec_fld(L):
    L = np.array(L)
    r = 1
    for i, la in enumerate(L):
        r += (la[0] - 1)*np.prod(L[0:i,1])
    return r

In [46]:

l1 = [9,1]
l2 = [9,1]

L = [l1,l2]

L = rec_fld(L)

L

17

In [47]:
CN = CapsNet()

In [48]:
a = CN.backbone.layers[0]
b = CN.primcaps.conv

a,b

a.kernel_size, a.stride
b.kernel_size, b.stride



((9, 9), (2, 2))

In [96]:
A = nn.Conv2d(in_channels=2, out_channels=3,kernel_size=5,stride=2,padding=0, bias=True)

count_parameters(A), 3*2*(5**2)+3, calc_params(ch_in=2, ch_out=3, kernel=5, bias=True)



(153, 153, 153)

In [94]:
def calc_params(ch_in, ch_out, kernel, bias=True):
    out = ch_in*ch_out*kernel**2
    if bias == True:
       out = out + ch_out
    return out

##### EFFCN

In [114]:
bb_effcn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(5, 5), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(3, 3), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=2, padding="valid"),
            nn.Conv2d(128, 128, kernel_size=9, stride=1, groups=128, padding="valid")
        )

In [115]:
count_parameters(bb_effcn)

140928

In [110]:
130432-130112, 130432-130304

(320, 128)

In [118]:
L_ef= [[5,1],[3,1],[3,1],[3,2],[9,1]]

rec_fld(L_ef)

27

##### EM Caps

In [ ]:
layer_em = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=A, kernel_size=(5, 5), stride=2, padding=2),

        )